In [2]:
import itertools

#### Reading the date

In [3]:
def read_data_sets():
    candidate_1 = {}
    transactions = 0
    dataset = []
    for i in range(1, 6, 1):
        with open("/mnt/Alice/ISI/SEM3/DataMining/Assignments/assignment3/dataset/DataSet{}.txt".format(i), "r") as f:
            for line in f:
                transac = []
                transactions += 1
                for word in line.split():
                    transac.append(word)
                    if word not in candidate_1.keys():
                        candidate_1[word] = 1
                    else:
                        count = candidate_1[word]
                        candidate_1[word] = count + 1
                dataset.append(transac)
    return candidate_1, dataset, transactions    

#### Utility Functions

In [4]:
def findsubsets(S, m):
    """
        Returns all the m element subsets of the set S
    """
    return set(itertools.combinations(S, m))

def has_infrequent_subset(candidate_set, frequent_k_1, k):
    """
        Returns True if pruning is required to remove the candidate_set 
                        if it is not present in the frequent k-1 itemset (frequent_k_1)
        else False
    """
    l = []
    l = findsubsets(candidate_set, k)
    for item in l: 
        s = []
        for l in item:
            s.append(l)
        s.sort()
        if s not in frequent_k_1:
            return True
    return False


#### Apriori Functions

In [5]:
def apriori_gen(frequent_k_1, k):
    """
        Computes the candidate set (candidate_k) using the frequent k-1 itemset (frequent_k_1) 
    """
    length = k
    candidate_k = [] 
    for list1 in frequent_k_1:
        for list2 in frequent_k_1:
            count = 0
            c = []
            if list1 != list2:
                while count < length-1:
                    if list1[count] != list2[count]:
                        break
                    else:
                        count += 1
                else:
                    if list1[length-1] < list2[length-1]:
                        for item in list1:
                            c.append(item)
                        c.append(list2[length-1])
                        if not has_infrequent_subset(c, frequent_k_1, k):
                            candidate_k.append(c) 
                            c = []
    return candidate_k

In [6]:
def frequent_itemsets(frequest_1, dataset):
    """
        Computes all the frequent_itemsets
    """
    k = 2
    frequent_k_1 = []
    frequent_k = []
    frequent_item_sets = []
    count = 0
    transactions = 0
    for item in frequest_1:
        frequent_k_1.append(item)
    while frequent_k_1 != []:
        candidate_k = []
        frequent_k = []
        candidate_k = apriori_gen(frequent_k_1, k-1)
#         print("-------------------------CANDIDATE {}-ITEMSET---------------------".format(k))
#         print("candidate_{}: {}".format(k, candidate_k))
#         print("------------------------------------------------------------------")
        for c in candidate_k:
            count = 0
            transactions = 0
            s = set(c)
            for T in dataset:
                transactions += 1
                t = set(T)
                if s.issubset(t) == True:
                    count += 1
            if (100 * count/transactions) >= support:
                c.sort()
                frequent_k.append(c)
        frequent_k_1 = []
#         print("-----------------------FREQUENT {}-ITEMSET------------------------".format(k))
#         print("frequent_{}: {}".format(k, frequent_k))
#         print("------------------------------------------------------------------")
        for l in frequent_k:
            frequent_k_1.append(l)
        k += 1
        if frequent_k != []:
            frequent_item_sets.append(frequent_k)
    return frequent_item_sets

In [7]:
def generate_association_rules(frequent_1, dataset):
    """
        Generates and print all the association rules with given support and confidence value
    """
    s = []
    r = []
    length = 0
    count = 1
    inc1 = 0
    inc2 = 0
    num = 1
    m = []
    L = frequent_itemsets(frequent_1, dataset)
    print("---------------------ASSOCIATION RULES------------------")
    print("RULES \t SUPPORT \t CONFIDENCE")
    print("--------------------------------------------------------")
    for list in L:
        for l in list:
            length = len(l)
            count = 1
            while count < length: 
                s = []
                r = findsubsets(l,count)
                count += 1
                for item in r:
                    inc1 = 0
                    inc2 = 0
                    s = []
                    m = []
                    for i in item:
                        s.append(i)
                    for T in dataset:
                        if set(s).issubset(set(T)) == True:
                            inc1 += 1
                        if set(l).issubset(set(T)) == True:
                            inc2 += 1
                    if 100*inc2/inc1 >= confidence:
                        for index in l:
                            if index not in s:
                                m.append(index)
                        print("Rule#  %d : %s ==> %s %d %d" %(num, s, m, 100*inc2/len(dataset), 100*inc2/inc1))
                        num += 1

In [8]:

support = 5
confidence = 10

candidate_1, dataset, transactions = read_data_sets()
print("Total Transactions : {}".format(transactions))
frequent_1 = []
for key in candidate_1:
    if (100 * candidate_1[key]/transactions) >= support:
        l = []
        l.append(key)
        frequent_1.append(l)

# print(dataset)

# print("-------------------------CANDIDATE 1-ITEMSET---------------------")
# print("candidate_1: {}".format(candidate_1))
# print("------------------------------------------------------------------")

# print("-------------------------FREQUENT 1-ITEMSET---------------------")
# print("frequest_1: {}".format(frequent_1))
# print("------------------------------------------------------------------")

generate_association_rules(frequent_1, dataset)

Total Transactions : 100
---------------------ASSOCIATION RULES------------------
RULES 	 SUPPORT 	 CONFIDENCE
--------------------------------------------------------
Rule#  1 : ['TV'] ==> ['cellphone'] 5 71
Rule#  2 : ['cellphone'] ==> ['TV'] 5 71
Rule#  3 : ['dishwasher'] ==> ['freezer'] 5 71
Rule#  4 : ['freezer'] ==> ['dishwasher'] 5 62
Rule#  5 : ['dishwasher'] ==> ['juicer'] 5 71
Rule#  6 : ['juicer'] ==> ['dishwasher'] 5 55
Rule#  7 : ['dishwasher'] ==> ['expresso'] 5 71
Rule#  8 : ['expresso'] ==> ['dishwasher'] 5 55
Rule#  9 : ['juicer'] ==> ['freezer'] 5 55
Rule#  10 : ['freezer'] ==> ['juicer'] 5 62
Rule#  11 : ['juicer'] ==> ['microwave'] 5 55
Rule#  12 : ['microwave'] ==> ['juicer'] 5 50
Rule#  13 : ['expresso'] ==> ['freezer'] 6 66
Rule#  14 : ['freezer'] ==> ['expresso'] 6 75
Rule#  15 : ['juicer'] ==> ['expresso'] 6 66
Rule#  16 : ['expresso'] ==> ['juicer'] 6 66
Rule#  17 : ['jewelry'] ==> ['microwave'] 5 55
Rule#  18 : ['microwave'] ==> ['jewelry'] 5 50
Rule#  19 : [